***GENERATED CODE FOR gettingdataaa PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'index', 'transformation_label': 'String Indexer'}], 'feature': 'index', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '137248', 'mean': '42258.7', 'stddev': '25234.01', 'min': '(40.507933', 'max': '999', 'missing': '0', 'distinct': '70250'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'index'}, {'feature_label': 'index', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('index')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Reporting Period', 'transformation_label': 'String Indexer'}], 'feature': 'Reporting Period', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': ' -71.949914)"', 'max': '2/28/2018', 'missing': '0', 'distinct': '1624'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Reporting Period'}, {'feature_label': 'Reporting Period', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Reporting Period')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Project Number', 'transformation_label': 'String Indexer'}], 'feature': 'Project Number', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '97572.2', 'stddev': '7985.24', 'min': '0107456-1', 'max': '99999', 'missing': '0', 'distinct': '68624'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Project Number'}, {'feature_label': 'Project Number', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Project Number')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'City', 'transformation_label': 'String Indexer'}], 'feature': 'City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': 'Accord', 'max': 'Youngsville', 'missing': '0', 'distinct': '1781'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'City'}, {'feature_label': 'City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'County', 'transformation_label': 'String Indexer'}], 'feature': 'County', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': 'Albany', 'max': 'Yates', 'missing': '0', 'distinct': '62'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'County'}, {'feature_label': 'County', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('County')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': 'NY', 'max': 'NY', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Sector', 'transformation_label': 'String Indexer'}], 'feature': 'Sector', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': 'Non-Residential', 'max': 'Residential', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Sector'}, {'feature_label': 'Sector', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Sector')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Program Type', 'transformation_label': 'String Indexer'}], 'feature': 'Program Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': 'Commercial/Industrial (Competitive)', 'max': 'Residential/Small Commercial', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Program Type'}, {'feature_label': 'Program Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Program Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Solicitation', 'transformation_label': 'String Indexer'}], 'feature': 'Solicitation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': 'GJGNY-OBR-10-349', 'max': 'Solar Pioneer Leasing 2014', 'missing': '0', 'distinct': '39'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Solicitation'}, {'feature_label': 'Solicitation', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Solicitation')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Electric Utility', 'transformation_label': 'String Indexer'}], 'feature': 'Electric Utility', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': 'Central Hudson Gas and Electric', 'max': 'Rochester Gas and Electric', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Electric Utility'}, {'feature_label': 'Electric Utility', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Electric Utility')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Purchase Type', 'transformation_label': 'String Indexer'}], 'feature': 'Purchase Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': 'Lease', 'max': 'Purchase', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Purchase Type'}, {'feature_label': 'Purchase Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Purchase Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Date Application Received', 'transformation_label': 'String Indexer'}], 'feature': 'Date Application Received', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': '1/10/2002', 'max': '9/9/2016', 'missing': '0', 'distinct': '3144'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Date Application Received'}, {'feature_label': 'Date Application Received', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Date Application Received')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Date Completed', 'transformation_label': 'String Indexer'}], 'feature': 'Date Completed', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': '1/1/2014', 'max': '9/9/2015', 'missing': '0', 'distinct': '2817'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Date Completed'}, {'feature_label': 'Date Completed', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Date Completed')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Project Status', 'transformation_label': 'String Indexer'}], 'feature': 'Project Status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': 'Complete', 'max': 'Pipeline', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Project Status'}, {'feature_label': 'Project Status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Project Status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Contractor', 'transformation_label': 'String Indexer'}], 'feature': 'Contractor', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': '0821 ABC, Inc', 'max': 'sun8 PDC LLC', 'missing': '0', 'distinct': '733'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Contractor'}, {'feature_label': 'Contractor', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Contractor')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Primary Inverter Manufacturer', 'transformation_label': 'String Indexer'}], 'feature': 'Primary Inverter Manufacturer', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': 'ABB', 'max': 'enphase', 'missing': '0', 'distinct': '197'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Primary Inverter Manufact...'}, {'feature_label': 'Primary Inverter Manufacturer', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Primary Inverter Manufacturer')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Primary Inverter Model Number', 'transformation_label': 'String Indexer'}], 'feature': 'Primary Inverter Model Number', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '362590.0', 'stddev': 'nan', 'min': '362590', 'max': 'blueplanet 5002xi (240V)', 'missing': '0', 'distinct': '873'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Primary Inverter Model Nu...'}, {'feature_label': 'Primary Inverter Model Number', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Primary Inverter Model Number')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Primary PV Module Manufacturer', 'transformation_label': 'String Indexer'}], 'feature': 'Primary PV Module Manufacturer', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': '1SOLTECH', 'max': 'aleo solar', 'missing': '0', 'distinct': '418'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Primary PV Module Manufac...'}, {'feature_label': 'Primary PV Module Manufacturer', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Primary PV Module Manufacturer')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PV Module Model Number', 'transformation_label': 'String Indexer'}], 'feature': 'PV Module Model Number', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': '1STH-245-WH', 'max': 'c-Si M60 NA 44117 265Wp', 'missing': '0', 'distinct': '1950'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PV Module Model Number'}, {'feature_label': 'PV Module Model Number', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PV Module Model Number')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Project Cost', 'transformation_label': 'String Indexer'}], 'feature': 'Project Cost', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': '$1,000,000.00 ', 'max': '$997,920.00 ', 'missing': '0', 'distinct': '36481'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Project Cost'}, {'feature_label': 'Project Cost', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Project Cost')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Incentive', 'transformation_label': 'String Indexer'}], 'feature': 'Incentive', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': '$0.00 ', 'max': '$999.00 ', 'missing': '0', 'distinct': '15388'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Incentive'}, {'feature_label': 'Incentive', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Incentive')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Remote Net Metering', 'transformation_label': 'String Indexer'}], 'feature': 'Remote Net Metering', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Remote Net Metering'}, {'feature_label': 'Remote Net Metering', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Remote Net Metering')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Affordable Solar', 'transformation_label': 'String Indexer'}], 'feature': 'Affordable Solar', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Affordable Solar'}, {'feature_label': 'Affordable Solar', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Affordable Solar')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Community Distributed Generation', 'transformation_label': 'String Indexer'}], 'feature': 'Community Distributed Generation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Community Distributed Gen...'}, {'feature_label': 'Community Distributed Generation', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Community Distributed Generation')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Green Jobs Green New York Participant', 'transformation_label': 'String Indexer'}], 'feature': 'Green Jobs Green New York Participant', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': 'Non-Residential', 'max': 'Residential', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Green Jobs Green New York...'}, {'feature_label': 'Green Jobs Green New York Participant', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Green Jobs Green New York Participant')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Location 1', 'transformation_label': 'String Indexer'}], 'feature': 'Location 1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '137248', 'mean': '', 'stddev': '', 'min': 'Accord, NY 12404', 'max': 'Youngsville, NY 12791', 'missing': '0', 'distinct': '2387'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Location 1'}, {'feature_label': 'Location 1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Location 1')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run gettingdataaaHooks.ipynb
try:
	#sourcePreExecutionHook()

	solarelectric = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Solar-electric.csv', 'filename': 'Solar-electric.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run gettingdataaaHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(solarelectric,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "index", "transformation_label": "String Indexer"}], "feature": "index", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "42258.7", "stddev": "25234.01", "min": "(40.507933", "max": "999", "missing": "0", "distinct": "70250"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "index"}, {"transformationsData": [{"feature_label": "Reporting Period", "transformation_label": "String Indexer"}], "feature": "Reporting Period", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": " -71.949914)\"", "max": "2/28/2018", "missing": "0", "distinct": "1624"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Reporting Period"}, {"transformationsData": [{"feature_label": "Project Number", "transformation_label": "String Indexer"}], "feature": "Project Number", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "97572.2", "stddev": "7985.24", "min": "0107456-1", "max": "99999", "missing": "0", "distinct": "68624"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Project Number"}, {"transformationsData": [{"feature_label": "City", "transformation_label": "String Indexer"}], "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "Accord", "max": "Youngsville", "missing": "0", "distinct": "1781"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "City"}, {"transformationsData": [{"feature_label": "County", "transformation_label": "String Indexer"}], "feature": "County", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "Albany", "max": "Yates", "missing": "0", "distinct": "62"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "County"}, {"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "NY", "max": "NY", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Zip Code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "137248", "mean": "11942.42", "stddev": "792.15", "min": "501", "max": "15086", "missing": "0"}, "updatedLabel": "Zip Code"}, {"transformationsData": [{"feature_label": "Sector", "transformation_label": "String Indexer"}], "feature": "Sector", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "Non-Residential", "max": "Residential", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Sector"}, {"transformationsData": [{"feature_label": "Program Type", "transformation_label": "String Indexer"}], "feature": "Program Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "Commercial/Industrial (Competitive)", "max": "Residential/Small Commercial", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Program Type"}, {"transformationsData": [{"feature_label": "Solicitation", "transformation_label": "String Indexer"}], "feature": "Solicitation", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "GJGNY-OBR-10-349", "max": "Solar Pioneer Leasing 2014", "missing": "0", "distinct": "39"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Solicitation"}, {"transformationsData": [{"feature_label": "Electric Utility", "transformation_label": "String Indexer"}], "feature": "Electric Utility", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "Central Hudson Gas and Electric", "max": "Rochester Gas and Electric", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Electric Utility"}, {"transformationsData": [{"feature_label": "Purchase Type", "transformation_label": "String Indexer"}], "feature": "Purchase Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "Lease", "max": "Purchase", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Purchase Type"}, {"transformationsData": [{"feature_label": "Date Application Received", "transformation_label": "String Indexer"}], "feature": "Date Application Received", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "1/10/2002", "max": "9/9/2016", "missing": "0", "distinct": "3144"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Date Application Received"}, {"transformationsData": [{"feature_label": "Date Completed", "transformation_label": "String Indexer"}], "feature": "Date Completed", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "1/1/2014", "max": "9/9/2015", "missing": "0", "distinct": "2817"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Date Completed"}, {"transformationsData": [{"feature_label": "Project Status", "transformation_label": "String Indexer"}], "feature": "Project Status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "Complete", "max": "Pipeline", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Project Status"}, {"transformationsData": [{"feature_label": "Contractor", "transformation_label": "String Indexer"}], "feature": "Contractor", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "0821 ABC, Inc", "max": "sun8 PDC LLC", "missing": "0", "distinct": "733"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Contractor"}, {"transformationsData": [{"feature_label": "Primary Inverter Manufacturer", "transformation_label": "String Indexer"}], "feature": "Primary Inverter Manufacturer", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "ABB", "max": "enphase", "missing": "0", "distinct": "197"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Primary Inverter Manufact..."}, {"transformationsData": [{"feature_label": "Primary Inverter Model Number", "transformation_label": "String Indexer"}], "feature": "Primary Inverter Model Number", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "362590.0", "stddev": "nan", "min": "362590", "max": "blueplanet 5002xi (240V)", "missing": "0", "distinct": "873"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Primary Inverter Model Nu..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total Inverter Quantity", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "137248", "mean": "8.95", "stddev": "10.95", "min": "1", "max": "800", "missing": "0"}, "updatedLabel": "Total Inverter Quantity"}, {"transformationsData": [{"feature_label": "Primary PV Module Manufacturer", "transformation_label": "String Indexer"}], "feature": "Primary PV Module Manufacturer", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "1SOLTECH", "max": "aleo solar", "missing": "0", "distinct": "418"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Primary PV Module Manufac..."}, {"transformationsData": [{"feature_label": "PV Module Model Number", "transformation_label": "String Indexer"}], "feature": "PV Module Model Number", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "1STH-245-WH", "max": "c-Si M60 NA 44117 265Wp", "missing": "0", "distinct": "1950"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PV Module Model Number"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total PV Module Quantity", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "137248", "mean": "77.62", "stddev": "380.76", "min": "1", "max": "14800", "missing": "0"}, "updatedLabel": "Total PV Module Quantity"}, {"transformationsData": [{"feature_label": "Project Cost", "transformation_label": "String Indexer"}], "feature": "Project Cost", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "$1,000,000.00 ", "max": "$997,920.00 ", "missing": "0", "distinct": "36481"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Project Cost"}, {"transformationsData": [{"feature_label": "Incentive", "transformation_label": "String Indexer"}], "feature": "Incentive", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "$0.00 ", "max": "$999.00 ", "missing": "0", "distinct": "15388"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Incentive"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total Nameplate kW DC", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "137248", "mean": "23.72", "stddev": "126.14", "min": "0.0", "max": "5799.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Total Nameplate kW DC"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Expected KWh Annual Production", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "137248", "mean": "31989.06", "stddev": "177122.12", "min": "164", "max": "6807591", "missing": "0"}, "updatedLabel": "Expected KWh Annual Produ..."}, {"transformationsData": [{"feature_label": "Remote Net Metering", "transformation_label": "String Indexer"}], "feature": "Remote Net Metering", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Remote Net Metering"}, {"transformationsData": [{"feature_label": "Affordable Solar", "transformation_label": "String Indexer"}], "feature": "Affordable Solar", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Affordable Solar"}, {"transformationsData": [{"feature_label": "Community Distributed Generation", "transformation_label": "String Indexer"}], "feature": "Community Distributed Generation", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Community Distributed Gen..."}, {"transformationsData": [{"feature_label": "Green Jobs Green New York Participant", "transformation_label": "String Indexer"}], "feature": "Green Jobs Green New York Participant", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "Non-Residential", "max": "Residential", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Green Jobs Green New York..."}, {"transformationsData": [{"feature_label": "Location 1", "transformation_label": "String Indexer"}], "feature": "Location 1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "137248", "mean": "", "stddev": "", "min": "Accord, NY 12404", "max": "Youngsville, NY 12791", "missing": "0", "distinct": "2387"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Location 1"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run gettingdataaaHooks.ipynb
try:
	#sinkPreExecutionHook()

	mavenmarketoutput = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/65d28f34b0c78b45eb628994/part-00000-eb0c013e-8006-4157-bff5-47dcf93dec4a-c000.csv', 'samplecount': 137248, 'originalcount': 169442, 'orignalKey': '/FileStore/platform/uploadedSourceFiles/Solar-electric.csv', 'pathOnly': '/MavenMarket', 'project_id': '65a7c21719e9407400dd7d45', 'parent_id': '65a7c21719e9407400dd7d45', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Zip Code', 'alias': 'Zip Code', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Total Inverter Quantity', 'alias': 'Total Inverter Quantity', 'type': 'numeric', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Total PV Module Quantity', 'alias': 'Total PV Module Quantity', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Total Nameplate kW DC', 'alias': 'Total Nameplate kW DC', 'type': 'real', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Expected KWh Annual Production', 'alias': 'Expected KWh Annual Production', 'type': 'numeric', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'index_stringindexer', 'alias': 'index_stringindexer', 'type': 'real', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Reporting Period_stringindexer', 'alias': 'Reporting Period_stringindexer', 'type': 'real', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Project Number_stringindexer', 'alias': 'Project Number_stringindexer', 'type': 'real', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'City_stringindexer', 'alias': 'City_stringindexer', 'type': 'real', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'County_stringindexer', 'alias': 'County_stringindexer', 'type': 'real', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'State_stringindexer', 'alias': 'State_stringindexer', 'type': 'numeric', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Sector_stringindexer', 'alias': 'Sector_stringindexer', 'type': 'numeric', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Program Type_stringindexer', 'alias': 'Program Type_stringindexer', 'type': 'numeric', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Solicitation_stringindexer', 'alias': 'Solicitation_stringindexer', 'type': 'real', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Electric Utility_stringindexer', 'alias': 'Electric Utility_stringindexer', 'type': 'real', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Purchase Type_stringindexer', 'alias': 'Purchase Type_stringindexer', 'type': 'numeric', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Date Application Received_stringindexer', 'alias': 'Date Application Received_stringindexer', 'type': 'real', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Date Completed_stringindexer', 'alias': 'Date Completed_stringindexer', 'type': 'real', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Project Status_stringindexer', 'alias': 'Project Status_stringindexer', 'type': 'numeric', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Contractor_stringindexer', 'alias': 'Contractor_stringindexer', 'type': 'real', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Primary Inverter Manufacturer_stringindexer', 'alias': 'Primary Inverter Manufacturer_stringindexer', 'type': 'real', 'position': '20'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Primary Inverter Model Number_stringindexer', 'alias': 'Primary Inverter Model Number_stringindexer', 'type': 'real', 'position': '21'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Primary PV Module Manufacturer_stringindexer', 'alias': 'Primary PV Module Manufacturer_stringindexer', 'type': 'real', 'position': '22'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'PV Module Model Number_stringindexer', 'alias': 'PV Module Model Number_stringindexer', 'type': 'real', 'position': '23'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Project Cost_stringindexer', 'alias': 'Project Cost_stringindexer', 'type': 'real', 'position': '24'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Incentive_stringindexer', 'alias': 'Incentive_stringindexer', 'type': 'real', 'position': '25'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Remote Net Metering_stringindexer', 'alias': 'Remote Net Metering_stringindexer', 'type': 'numeric', 'position': '26'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Affordable Solar_stringindexer', 'alias': 'Affordable Solar_stringindexer', 'type': 'numeric', 'position': '27'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Community Distributed Generation_stringindexer', 'alias': 'Community Distributed Generation_stringindexer', 'type': 'numeric', 'position': '28'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Green Jobs Green New York Participant_stringindexer', 'alias': 'Green Jobs Green New York Participant_stringindexer', 'type': 'numeric', 'position': '29'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Location 1_stringindexer', 'alias': 'Location 1_stringindexer', 'type': 'real', 'position': '30'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'index', 'alias': 'index', 'type': 'String', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Reporting Period', 'alias': 'Reporting Period', 'type': 'String', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Project Number', 'alias': 'Project Number', 'type': 'String', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'City', 'alias': 'City', 'type': 'String', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'County', 'alias': 'County', 'type': 'String', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'State', 'alias': 'State', 'type': 'String', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Zip Code', 'alias': 'Zip Code', 'type': 'real', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Sector', 'alias': 'Sector', 'type': 'String', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Program Type', 'alias': 'Program Type', 'type': 'String', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Solicitation', 'alias': 'Solicitation', 'type': 'String', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Electric Utility', 'alias': 'Electric Utility', 'type': 'String', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Purchase Type', 'alias': 'Purchase Type', 'type': 'String', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Date Application Received', 'alias': 'Date Application Received', 'type': 'date', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Date Completed', 'alias': 'Date Completed', 'type': 'date', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Project Status', 'alias': 'Project Status', 'type': 'String', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Contractor', 'alias': 'Contractor', 'type': 'String', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Primary Inverter Manufacturer', 'alias': 'Primary Inverter Manufacturer', 'type': 'String', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Primary Inverter Model Number', 'alias': 'Primary Inverter Model Number', 'type': 'String', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Total Inverter Quantity', 'alias': 'Total Inverter Quantity', 'type': 'real', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Primary PV Module Manufacturer', 'alias': 'Primary PV Module Manufacturer', 'type': 'String', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'PV Module Model Number', 'alias': 'PV Module Model Number', 'type': 'String', 'position': '20'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Total PV Module Quantity', 'alias': 'Total PV Module Quantity', 'type': 'real', 'position': '21'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Project Cost', 'alias': 'Project Cost', 'type': 'String', 'position': '22'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Incentive', 'alias': 'Incentive', 'type': 'String', 'position': '23'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Total Nameplate kW DC', 'alias': 'Total Nameplate kW DC', 'type': 'real', 'position': '24'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Expected KWh Annual Production', 'alias': 'Expected KWh Annual Production', 'type': 'real', 'position': '25'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Remote Net Metering', 'alias': 'Remote Net Metering', 'type': 'String', 'position': '26'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Affordable Solar', 'alias': 'Affordable Solar', 'type': 'String', 'position': '27'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Community Distributed Generation', 'alias': 'Community Distributed Generation', 'type': 'String', 'position': '28'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Green Jobs Green New York Participant', 'alias': 'Green Jobs Green New York Participant', 'type': 'String', 'position': '29'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Location 1', 'alias': 'Location 1', 'type': 'String', 'position': '30'}], 'server': 'https://ml.colaberry.com:443', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'FileNmae.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-af4c1599-ddc1-4e4e-b813-768a88b2251e-c000.csv', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
